# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   583.843040 -0.797342  1234  583.045697  584.640382  1166.091395   
6   202.337240 -0.729127  1234  201.608114  203.066367   403.216227   
7    17.081737  0.467132  1234   17.548869   16.614606    35.097738   
8   492.757768  0.018676  1234  492.776443  492.739092   985.552887   
9   667.167306 -0.980435  1234  666.186871  668.147741  1332.373741   
10  938.738971  0.747193  1234  939.486164  937.991778  1878.972329   
11  964.042355  0.049400  1234  964.091756  963.992955  1928.183512   
12    0.368256  0.094693  1234    0.462950    0.273563     0.925899   
13  217.958981  0.511284  1234  218.470266  217.447697   436.940531   
14  275.664771  0.972331  1234  276.637102  274.692441   553.274204   

              f  
5   1169.280764  
6    406.132734  
7     33.229211  
8    985.478184  
9   1336.295482  
10  1875.983556  
11  1927.985910  
12     0.547126  
13   434.895394  
14   549.384882

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -583.045697   584.640382
6   -201.608114   203.066367
7    -17.548869    16.614606
8   -492.776443   492.739092
9   -666.186871   668.147741
10  -939.486164   937.991778
11  -964.091756   963.992955
12    -0.462950     0.273563
13  -218.470266   217.447697
14  -276.637102   274.692441

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -583.045697   584.640382
6   -201.608114   203.066367
7    -17.548869    16.614606
8   -492.776443   492.739092
9   -666.186871   668.147741
10  -939.486164   937.991778
11  -964.091756   963.992955
12    -0.462950     0.273563
13  -218.470266   217.447697
14  -276.637102   274.692441

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-5.83045697e+02,  5.84640382e+02],
       [-2.01608114e+02,  2.03066367e+02],
       [-1.75488690e+01,  1.66146056e+01],
       [-4.92776443e+02,  4.92739092e+02],
       [-6.66186871e+02,  6.68147741e+02],
       [-9.39486164e+02,  9.37991778e+02],
       [-9.64091756e+02,  9.63992955e+02],
       [-4.62949713e-01,  2.73562879e-01],
       [-2.18470266e+02,  2.17447697e+02],
       [-2.76637102e+02,  2.74692441e+02]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1164.091395  -1169.280764
6     401.216227   -406.132734
7      33.097738    -33.229211
8     983.552887   -985.478184
9    1330.373741  -1336.295482
10   1876.972329  -1875.983556
11   1926.183512  -1927.985910
12     -1.074101     -0.547126
13    434.940531   -434.895394
14    551.274204   -549.384882

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12        True        True      True
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf